In [1]:
!pip install pandas
!pip install numpy

In [2]:
dtypes = {
    'DayOfWeek':                       int,
    'FlightDate':                      'string',
    'IATA_CODE_Reporting_Airline':     'string',
    'Tail_Number':                     'string',
    'Flight_Number_Reporting_Airline': int,
    'OriginAirportID':                 int,
    'Origin':                          'string',
    'OriginState':                     'string',
    'DestAirportID':                   int,
    'Dest':                            'string',
    'DestState':                       'string',
    'CRSDepTime':                      int,
    'DepTime':                         int,
    'DepDelayMinutes':                 int,
    'CRSArrTime':                      int,
    'ArrTime':                         int,
    'ArrDelayMinutes':                 int,
    'Cancelled':                       int,
    'CancellationCode':                'string',
    'Diverted':                        int,
    'CRSElapsedTime':                  int,
    'ActualElapsedTime':               int,
    'AirTime':                         int,
    'CarrierDelay':                    int,
    'WeatherDelay':                    int,
    'NASDelay':                        int,
    'SecurityDelay':                   int,
    'LateAircraftDelay':               int,
    'CRSDepHour':                      int,
    'OriginTz':                        'string',
    'DestTz':                          'string',
    'CRSArrDateTime':                  'string',
    'CRSArrDate':                      'string',
    'CRSArrHour':                      int,
    'o_temperature_2m':                float,
    'o_relative_humidity_2m':          float,
    'o_dew_point_2m':                  float,
    'o_apparent_temperature':          float,
    'o_precipitation':                 float,
    'o_rain':                          float,
    'o_snowfall':                      float,
    'o_snow_depth':                    float,
    'o_weather_code':                  float,
    'o_surface_pressure':              float,
    'o_cloud_cover':                   float,
    'o_cloud_cover_low':               float,
    'o_cloud_cover_mid':               float,
    'o_cloud_cover_high':              float,
    'o_wind_speed_10m':                float,
    'o_wind_speed_100m':               float,
    'o_wind_gusts_10m':                float,
    'o_shortwave_radiation':           float,
    'o_direct_radiation':              float,
    'o_diffuse_radiation':             float,
    'o_direct_normal_irradiance':      float,
    'o_terrestrial_radiation':         float,
    'o_airport':                       'string',
    'o_day':                           'string',
    'o_hour':                          int,
    'd_temperature_2m':                float,
    'd_relative_humidity_2m':          float,
    'd_dew_point_2m':                  float,
    'd_apparent_temperature':          float,
    'd_precipitation':                 float,
    'd_rain':                          float,
    'd_snowfall':                      float,
    'd_snow_depth':                    float,
    'd_weather_code':                  float,
    'd_surface_pressure':              float,
    'd_cloud_cover':                   float,
    'd_cloud_cover_low':               float,
    'd_cloud_cover_mid':               float,
    'd_cloud_cover_high':              float,
    'd_wind_speed_10m':                float,
    'd_wind_speed_100m':               float,
    'd_wind_gusts_10m':                float,
    'd_shortwave_radiation':           float,
    'd_direct_radiation':              float,
    'd_diffuse_radiation':             float,
    'd_direct_normal_irradiance':      float,
    'd_terrestrial_radiation':         float,
    'd_airport':                       'string',
    'd_day':                           'string',
    'd_hour':                          int
}
files = [
    'data/weather-joined/full-w-2017.csv',
    'data/weather-joined/full-w-2018.csv',
    'data/weather-joined/full-w-2019.csv',
]

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import scienceplots
import seaborn as sns

import pickle

In [4]:
# plot style
plt.style.use(['science','no-latex'])

# plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams['savefig.format'] = 'eps'
plt.rcParams['figure.figsize'] = (6, 3)

# display all rows and columns when printing in the notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
df = pd.read_csv('data/weather-joined/full-w-2017.csv')

/tmp/ipykernel_312950/3735653777.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/weather-joined/full-w-2017.csv')


In [118]:
def clean_data(df):
    # convert string to datetime
    datetime_cols = ['FlightDate', 'CRSArrDateTime', 'CRSArrDate', 'o_day', 'd_day']
    
    for col in datetime_cols:
        df[col] = pd.to_datetime(df[col])

    # drop rows that have cancelled or diverted flights
    df = df.drop(df[(df['Cancelled'] == 1) | (df['Diverted'] == 1)].index)

    # fill 0s for columns that make sense
    cols = ["CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay", "o_snow_depth", "d_snow_depth"]
    df[cols] = df[cols].fillna(value=0)

    df = df.drop('CancellationCode', axis=1)
    df = df.dropna()
    
    return df

In [119]:
df = clean_data(df)

/tmp/ipykernel_133879/1311960660.py:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[col] = pd.to_datetime(df[col])


In [232]:
def cluster_airports(airport_df, features, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    airport_df['cluster_labels'] = kmeans.fit_predict(airport_df[features])
    return airport_df

def add_cluster_labels(df, features, num_clusters):
    airport_df = pd.read_csv('latitude_longitude.csv')
    airport_df = cluster_airports(airport_df, features, num_clusters)
    clustered = pd.merge(df, airport_df, left_on=['Origin'], right_on=['iata_code'], how='left')
    return clustered

In [229]:
df = add_cluster_labels(df, ['latitude_deg', 'longitude_deg'], 30)

/home/shared/code/my-venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [230]:
df.columns

Index(['DayOfWeek', 'FlightDate', 'IATA_CODE_Reporting_Airline', 'Tail_Number',
       'Flight_Number_Reporting_Airline', 'OriginAirportID', 'Origin',
       'OriginState', 'DestAirportID', 'Dest', 'DestState', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'Cancelled', 'Diverted', 'CRSElapsedTime',
       'ActualElapsedTime', 'AirTime', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'CRSDepHour',
       'OriginTz', 'DestTz', 'CRSArrDateTime', 'CRSArrDate', 'CRSArrHour',
       'o_temperature_2m', 'o_relative_humidity_2m', 'o_dew_point_2m',
       'o_apparent_temperature', 'o_precipitation', 'o_rain', 'o_snowfall',
       'o_snow_depth', 'o_weather_code', 'o_surface_pressure', 'o_cloud_cover',
       'o_cloud_cover_low', 'o_cloud_cover_mid', 'o_cloud_cover_high',
       'o_wind_speed_10m', 'o_wind_speed_100m', 'o_wind_gusts_10m',
       'o_shortwave_radiation', 'o_direct_radiation', 'o_diff

In [123]:
# Balance the dataset by sampling equal number of non-delay records as delay records
def balance_dataset(df):
    delay_mask = (df['NASDelay'] > 0) | (df['WeatherDelay'] > 0)
    delay_df = df[delay_mask]
    non_delay_df = df[~delay_mask]
    
    num_delay_records = delay_df.shape[0]
    df = pd.concat([delay_df, non_delay_df.sample(num_delay_records, replace=False)])
    return df

In [181]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

def scale_df(df):
    columns_to_scale = df.columns #['o_temperature_2m', 'o_relative_humidity_2m', 'o_dew_point_2m', 'o_apparent_temperature', 'o_precipitation', 'o_rain', 'o_snowfall', 'o_snow_depth', 'o_surface_pressure', 'o_cloud_cover', 'o_cloud_cover_low', 'o_cloud_cover_mid', 'o_cloud_cover_high', 'o_wind_speed_10m', 'o_wind_speed_100m', 'o_wind_gusts_10m', 'o_shortwave_radiation', 'o_direct_radiation', 'o_diffuse_radiation', 'o_direct_normal_irradiance', 'o_terrestrial_radiation']

    scaler = StandardScaler()

    scaled_data = df.copy()
    scaled_data[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

    return scaled_data

In [206]:
# Note: Perform PCA only after standardization, or else some features will dominate.
def dim_reduction(df, num_components):
    num_components = min(num_components, *df.shape)
    pca = PCA(n_components=num_components)
    pca.fit(df)
    X_pca = pca.transform(df)
    return X_pca

In [207]:
# Give full df, not sampled.
def train_cluster(df, models, features):
    model_errors = []
    clusters = df.groupby('cluster_labels')
    for cluster, cluster_df in clusters:
        pop = cluster_df.shape[0]
        cluster_df = balance_dataset(cluster_df)
        X = cluster_df[features]
        Y = cluster_df['WeatherDelay'] + cluster_df['NASDelay']
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, shuffle=True)
        model = Model(*model_params)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test).clip(min=0, max=120)
        rmse = mean_squared_error(Y_test, Y_pred, squared=False)
        mae = mean_absolute_error(Y_test, Y_pred)
        rscore = r2_score(Y_test, Y_pred)

        bool_pred = Y_pred >= 15
        bool_test = Y_test >= 15
        accuracy = np.sum(bool_pred == bool_test)/len(bool_pred)
        tn, fp, fn, tp = confusion_matrix(bool_test, bool_pred).ravel()/len(bool_pred)
        fpr = fp/(fp+tn)
        tpr = tp/(tp+fn)
        
        # without 0 predictions
        # X_test = X_test[Y_test==0]
        # Y_test = Y_test[Y_test==0]
        # if X_test.shape[0] > 0:
        #     Y_pred = model.predict(X_test).clip(min=0, max=120)
        #     rmse0 = mean_squared_error(Y_test, Y_pred, squared=False)
        #     mae0 = mean_absolute_error(Y_test, Y_pred)
        #     rscore0 = r2_score(Y_test, Y_pred)
        # else:
        #     rmse0 = 0
        #     mae0 = 0
        #     rscore0 = 0
        # print(f'Cluster{cluster} RMSE: {rmse}')
        # print(f'Cluster{cluster} MAE: {mae}')
        # print(f'Cluster{cluster} R2: {mae}')
        
        model_errors.append((cluster, pop, cluster_df.shape[0], rmse, mae, rscore, accuracy, fpr, tpr))
        # pickle.dump(model, open(f'model-{cluster}.sav', 'wb'))
    return model_errors

def predict_cluster(df, features):
    model_errors = []
    clusters = df.groupby('cluster_labels')
    for cluster, cluster_df in clusters:
        X_test = cluster_df[features]
        loaded_model = pickle.load(open(f'model-{cluster}.sav', 'rb'))
        Y_pred = loaded_model.predict(X_test).clip(min=0, max=120)
        rmse = mean_squared_error(Y_test, Y_pred, squared=False)
        mae = mean_absolute_error(Y_test, Y_pred)
        model_errors.append((cluster, rmse, mae))
    return model_errors

In [207]:
# Give full df, not sampled.
def train_cluster(df, models, features):
    model_errors = []
    clusters = df.groupby('cluster_labels')
    for cluster, cluster_df in clusters:
        pop = cluster_df.shape[0]
        cluster_df = balance_dataset(cluster_df)
        X = cluster_df[features]
        Y = cluster_df['WeatherDelay'] + cluster_df['NASDelay']
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, shuffle=True)
        model = Model(*model_params)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test).clip(min=0, max=120)
        rmse = mean_squared_error(Y_test, Y_pred, squared=False)
        mae = mean_absolute_error(Y_test, Y_pred)
        rscore = r2_score(Y_test, Y_pred)

        bool_pred = Y_pred >= 15
        bool_test = Y_test >= 15
        accuracy = np.sum(bool_pred == bool_test)/len(bool_pred)
        tn, fp, fn, tp = confusion_matrix(bool_test, bool_pred).ravel()/len(bool_pred)
        fpr = fp/(fp+tn)
        tpr = tp/(tp+fn)
        
        # without 0 predictions
        # X_test = X_test[Y_test==0]
        # Y_test = Y_test[Y_test==0]
        # if X_test.shape[0] > 0:
        #     Y_pred = model.predict(X_test).clip(min=0, max=120)
        #     rmse0 = mean_squared_error(Y_test, Y_pred, squared=False)
        #     mae0 = mean_absolute_error(Y_test, Y_pred)
        #     rscore0 = r2_score(Y_test, Y_pred)
        # else:
        #     rmse0 = 0
        #     mae0 = 0
        #     rscore0 = 0
        # print(f'Cluster{cluster} RMSE: {rmse}')
        # print(f'Cluster{cluster} MAE: {mae}')
        # print(f'Cluster{cluster} R2: {mae}')
        
        model_errors.append((cluster, pop, cluster_df.shape[0], rmse, mae, rscore, accuracy, fpr, tpr))
        # pickle.dump(model, open(f'model-{cluster}.sav', 'wb'))
    return model_errors

def predict_cluster(df, features):
    model_errors = []
    clusters = df.groupby('cluster_labels')
    for cluster, cluster_df in clusters:
        X_test = cluster_df[features]
        loaded_model = pickle.load(open(f'model-{cluster}.sav', 'rb'))
        Y_pred = loaded_model.predict(X_test).clip(min=0, max=120)
        rmse = mean_squared_error(Y_test, Y_pred, squared=False)
        mae = mean_absolute_error(Y_test, Y_pred)
        model_errors.append((cluster, rmse, mae))
    return model_errors

In [208]:
temp_col_list = df.select_dtypes(['number']).columns.tolist()
unused_col_list = [
        'DayOfWeek',
        'Flight_Number_Reporting_Airline',
        'OriginAirportID',
        'DestAirportID',
        'CRSDepTime',
        'DepTime',
        'DepDelayMinutes',
        'CRSArrTime',
        'ArrTime',
        'ArrDelayMinutes',
        'Cancelled',
        'Diverted',
        # 'CRSElapsedTime',
        'ActualElapsedTime',
        'AirTime',
        'CarrierDelay',
        'WeatherDelay',
        'NASDelay',
        'SecurityDelay',
        'LateAircraftDelay',
        'CRSDepHour',
        'CRSArrHour',
        'o_weather_code',
        'o_hour',
        'd_weather_code',
        'd_hour',
        'latitude_deg',
        'longitude_deg'
    ]

In [209]:
features = [x for x in temp_col_list if x not in unused_col_list]
features

['CRSElapsedTime',
 'o_temperature_2m',
 'o_relative_humidity_2m',
 'o_dew_point_2m',
 'o_apparent_temperature',
 'o_precipitation',
 'o_rain',
 'o_snowfall',
 'o_snow_depth',
 'o_surface_pressure',
 'o_cloud_cover',
 'o_cloud_cover_low',
 'o_cloud_cover_mid',
 'o_cloud_cover_high',
 'o_wind_speed_10m',
 'o_wind_speed_100m',
 'o_wind_gusts_10m',
 'o_shortwave_radiation',
 'o_direct_radiation',
 'o_diffuse_radiation',
 'o_direct_normal_irradiance',
 'o_terrestrial_radiation',
 'd_temperature_2m',
 'd_relative_humidity_2m',
 'd_dew_point_2m',
 'd_apparent_temperature',
 'd_precipitation',
 'd_rain',
 'd_snowfall',
 'd_snow_depth',
 'd_surface_pressure',
 'd_cloud_cover',
 'd_cloud_cover_low',
 'd_cloud_cover_mid',
 'd_cloud_cover_high',
 'd_wind_speed_10m',
 'd_wind_speed_100m',
 'd_wind_gusts_10m',
 'd_shortwave_radiation',
 'd_direct_radiation',
 'd_diffuse_radiation',
 'd_direct_normal_irradiance',
 'd_terrestrial_radiation',
 'elevation_m',
 'cluster_labels']

In [218]:
errs = train_cluster(df, DecisionTreeRegressor, {}, features)

/home/shared/code/my-venv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/tmp/ipykernel_133879/1291648740.py:23: RuntimeWarning: invalid value encountered in scalar divide
  tpr = tp/(tp+fn)


In [219]:
err_df = pd.DataFrame(errs, columns=['cluster', 'cluster_pop', 'model_pop', 'rmse', 'mae', 'r2', 'accuracy', 'fpr', 'tpr'])

In [220]:
err_df

,cluster,cluster_pop,model_pop,rmse,mae,r2,accuracy,fpr,tpr
0,0,168569,28210,49.374706,22.652685,-0.160207,0.627282,0.309432,0.514217
1,1,354427,82988,33.782763,16.151039,-0.191245,0.662687,0.257968,0.467105
2,2,364,58,11.484773,8.400000,-1.531184,0.650000,0.235294,0.000000
3,3,21483,3260,24.411646,12.742565,-0.159482,0.613383,0.282993,0.390029
4,4,104686,18804,52.300562,22.489446,-0.142071,0.635514,0.288873,0.465445
5,5,194509,44826,38.856432,19.544987,-0.158257,0.634760,0.292268,0.480185
6,6,104184,6124,15.647812,9.535873,-0.669205,0.684315,0.217975,0.409434
7,7,245353,52296,42.193869,18.212076,-0.114181,0.654131,0.267429,0.491911
8,8,118,2,120.000000,120.000000,NaN,0.000000,1.000000,NaN
9,9,414366,96750,45.407231,22.374937,-0.031933,0.666907,0.276828,0.557876


In [221]:
err_df[err_df['model_pop'] > 500].describe()

,cluster,cluster_pop,model_pop,rmse,mae,r2,accuracy,fpr,tpr
count,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000
mean,15.076923,214541.846154,44678.769231,41.876593,18.909175,-0.259576,0.652217,0.272114,0.471131
std,8.826882,193381.822499,43966.779350,11.034007,3.953156,0.469364,0.024891,0.031915,0.069113
min,0.000000,3183.000000,524.000000,15.647812,9.535873,-2.460336,0.613383,0.160550,0.260870
25%,7.500000,53373.000000,6150.500000,38.916487,17.508527,-0.189760,0.634948,0.258945,0.453130
50%,15.500000,161714.000000,25418.000000,43.358037,20.117587,-0.150164,0.649711,0.281877,0.487289
75%,21.750000,373394.500000,82205.500000,48.919251,21.951614,-0.103684,0.661624,0.292094,0.521156
max,29.000000,731588.000000,151882.000000,65.526349,22.803900,0.025926,0.718693,0.309432,0.557876
